# Aperçu de quelques librairies pertinentes en python par l'exemple: les premiers blocks d'un moteur de recherche

Dans cet exemple, nous allons concevoir une application simple capable:
<ul>
<li> d'analyser le contenu de pages web
<li> distinguer le contenu textuel du reste des informations
<li> réaliser quelques analyses sémantiques de routine
<li> encoder les données dans un format aisément calculable pour la suite
<li> (opt.) analyser les images
</ul>

Cet exercice sera l'occasion d'utiliser certaines des principales librairies utilisées en Data Science. 
<ul>
<li> BeautifulSoup, un parser HTML <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">doc Officielle</a>
<li> Numpy, une librairie de calcul matriciel <a href="http://www.numpy.org/">doc Officielle</a>
<li> NLTK, une librairie pour le traitement automatisé du langage <a href="http://www.nltk.org/">doc Officielle</a>
<li> Pandas, une librairie orientée analyse de données (fera un peu penser à R pour les initiés) <a href="http://pandas.pydata.org/">doc Officielle</a>
<li> Seaborn, un outil de visualisation statistique <a href="https://seaborn.pydata.org/">doc Officielle</a>
<li> ainsi que diverses librairies pour optimiser notre script, lire et écrire les données en divers formats (CSV, JSON, etc)
</ul>

Commençons par importer tout ce joli petit monde

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
from bs4 import BeautifulSoup

## Le Temps, cette ressource précieuse
Avant de commencer, j'aimerais débunker une croyance assez répandue, même au sein des devs, selon laquelle python serait, pour des raisons structurelles et inaccessibles, lent lourd et inefficient pour des applications à taille réelle. Deux niveau de réponses

Premièrement, ce problème n'est pas propre à Python mais concerne l'ensemble des langages de haut degré d'abstraction (trad "vous voulez du rapide, codez en C")

Deuxièmement, le problème ne provient pas tant du langage en lui-même que du fait que les opérations précises de l'interpréteurs ne nous étant pas accessibles, deux <b>implémentations différentes</b> de la même tâche peuvent demander un temps assez différent alors même qu'elle paraissent "conceptuellement identiques". 

Prenons un exemple en nous dotant tout d'abord d'un moyen de benchmarker nos opérations avant de chercher à optimiser notre code

In [2]:
from time import time
import math

def Benchmark(fonction): 
    start = time()
    fonction
    end = time()
    return (end - start)

In [ ]:
dataTest = np.random.rand(600000,3) # on génère 600000*3 points de données x,y,z (vecteur 2-D)
dataTestAsList = dataTest.tolist()  # on en fait une liste pour les besoins de l'exemple
print(dataTest)

Prenons une fonction un peu lourde à calculer pour comparer deux approches: l'une itérative, l'autre vectorisée

$f(x,y,z)= \frac{x^{2}-\sqrt{y}}{\sqrt{x+y}} \times z$

In [ ]:
resultsList = [] # on crée une liste vide
start = time() # on lance le chrono
None #Code à compléter

end = time() # on arrête le chrono

print("temps nécessaire: ",end - start)
print(resultsList[:15])          # on imprime les 15 premiers résultats
print(len(resultsList))

In [ ]:
x = dataTest[:,0] # premier vecteur = première colonne de la matrice
y = dataTest[:,1] # second vecteur = seconde colonne de la matrice
z = dataTest[:,2]

start = time() # on lance le chrono
results = None #Code à compléter
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)
print(results)
print(len(results))

On voit assez aisément que, même pour ce petit dataset de trois valeurs x y z de 600000 points de donnée, l'<b>approche iterative</b> par les bloucles prend plus de 10 fois plus de temps que l'<b>approche vectorielle</b> permise par numpy pour le <b>même calcul</b>. Il existe une correpondance intuitive entre:
<ul>
<li> $a*b$ et np.multiply(a,b)
<li> $a**2$ et np.power(x,2)
<li> $a+b$ et np.sum((x,y), axis=0)
</ul>

Toutefois, lorsque a et b peuvent s'apparenter à des <b>vecteurs</b> de même longueur, du point de vue de l'interpréteur, cela fait une ÉNORME différence de réaliser les opérations de manière vectorielle plutôt qu'élément par élément (<i>element-wise</i>). La leçon a retenir est double: 
<ul>
<li> aussi souvent que possible, si vous le pouvez, <b>évitez de recourir à des boucles lorsque vous avez des données structurées</b>
<li> ce n'est pas parceque vous avec l'impression de décomposer une opération dans votre code que vous facilitez la tâche de l'interpréteur (il y a une couche d'abstraction entre son monde et le vôtre)
</ul>

Nous verrons en outre que la vectorisation n'est pas le seul moyen d'optimiser un code

## Constitution de notre set de données

Imaginez qu'un client (twitter, pour ne pas le nommer) vous commande un plugin web pour analyser toutes les pages indiquées en lien sur une chat box de son site propre. En gros, rien de bien sorcier, il veut pouvoir savoir rapidement de quoi la page parle, s'il y a des images, le ton général de l'article, s'il s'agit d'un page plutôt verbeuse, si elle parle de l'actualité ou au contraire d'événements anciens dans le but éventuel d'anlyser tout cela plus tard pour une analyse de marchée. Bref, un moyen de savoir ce dont il retourne sans avoir à tout lire.

Tout ce dont vous disposez, c'est d'un fichier CSV dont chaque ligne contient un post sur la chat box. 

On serait tenté d'importer ces données sous la forme d'une liste de chaîne de caractères sur laquelle appliquer les fonctions que nous allons importer ou créer. Ce serait toutefois ignorer notre précédente remarque sur les données structurées et la vectorisation. 

### Structuration et manipulation de données avec Pandas

Pour exploiter au mieux la structuration des données, nous allons les importer sous la forme d'un tableau de données (<i>dataframe</i>) avec la librairie pandas



In [ ]:
data = pd.read_csv("chatbox.csv",sep=',')
print(type(data))

print(data.head())

$data$ est un objet de type particulier, doté de ses propres attributs et méthodes (ex. $head()$ pour avoir les premières lignes du tableau). Par défaut, $read_csv$ utilise la première ligne comme le nom des colonnes. Regardons ces données de plus près. 

In [ ]:
print("champs de données (colonnes): ",list(data))
print("nombre de lignes: ", len(data))
print(data.describe())

Il est assez simple de sélectionner des colonnes spécifiques ou de spliter un tableau en fonction de la valeur dans un champ donné. Prenons le cas des messages "likés" au moins une fois. 

In [ ]:
data["liked"] # la colonne liked du tableau
dataLiked = data[(data["liked"]!=0)] # partie des données dont liked diffère de zero

print("nombre de lignes: ", len(dataLiked))

114223 messages pour seulement 25252 likés. Ça donne une idée des eusses et coutumes de cette chatbox. La methode de subseting peut paraître déconcertante au premier abord avec cet enchassement $[()]$. Retenez simplement que, en pandas, $[]$ permet de subseter, $()$ énonce une condition. 

Mainenant que nous avons vu les données, il faut que nous extrayons les urls du champ $"text"$. Pour cela, pas besoin de tirer la mouche au canon, une simple analyse d'expression régulière (oui, du Regex !) suffit. Une url n'est jamais rien d'autre qu'
<ul>
<li> une chaine de caractère
<li> ne contenant pas d'espace
<li> commençant par "http"
</ul>

Comme précédemment, nous pourrions extraire chaque ligne de $data$, analyser chaque $text$ à l'aide d'une bloucle: 

In [ ]:
import re # librairie de référence pour le Regex

Text = data["text"].tolist() # on crée une liste 
print(type(Text))

urls = []
start = time() # on lance le chrono
for i in Text:
    url = None #Code à compléter
    if url != '':
        None #Code à compléter
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

print(urls[:15])


Essayons maintenant avec une methode built-in. Nous introduisons une nouvelle colonne de données appelée $urls$

In [ ]:
start = time() # on lance le chrono
data['urls'] = data['text'].str. None #CODE A COMPLÉTER
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

print(data.head())

Maintenant que nous avons notre champ $"urls"$, concentrons nous exclusivement sur les posts qui renvoient à des liens externes. Le principe reste le même à quelques ajustement près. $"urls"$ contient pour l'instant des listes de <b>str</b> ce qui risque de nous faire un peu chier pour la suite. Nous allons considérer que seul le premier lien nous intéresse et changer le type des données de $"urls"$ de <b>list</b> à <b>str</b>. Vérifions d'abord qu'il s'agit bien de listes:  

In [ ]:
print(type(data["urls"]))
print(data["urls"].head())

Bon, ça risque d'être un peu chiant. Essayons autrement

In [ ]:
data['urls'] = data['urls'].astype(str)  # on en fait une chaîne de caractère et on va virer les caractères problématiques

data['urls'] = None #Code à compléter # on vire le premier caractère ([)
data['urls'] = None #Code à compléter # on vire le dernier caractère (])
data['urls']

C'est mieux, nous avons maintenant des champs de données vides => nous allons pouvoir faire un subsetting

In [ ]:
dataUrl = data[(data['urls']!='')]
print(dataUrl['urls'].head(25))
print(len(dataUrl['urls']))

Affinons encore. Pour prendre uniquement la première url, il suffit de virer ce que nous avons après "," (split)

In [ ]:
dataUrl['urls'].astype(str)
print(dataUrl['urls'].head(25))
dataUrl['urls'] = None #Code à compléter
print(dataUrl['urls'])

Dernier coup de reins, on vire les " et les ' (replace)

In [ ]:
dataUrl['urls'] = None #CODE A COMPLÉTER
dataUrl['urls'] = None #CODE A COMPLÉTER

print(dataUrl['urls'].head(25))

Bien, nous avons nos urls bien propres. On jette un dernier regard à notre set de données final avant de le sauvegarder dans un fichier JSON. 

In [ ]:
print(dataUrl.head(30))
dataUrl.to_json("talkboxUrl.json")


## Raffinement de données: identification des véritables urls

Pour développer nos fonctions, nous allons prendre quelques une des urls du dataset et les convertir en liste

In [ ]:
urlsTest = None #CODE A COMPLÉTER
print(urlsTest)

In [ ]:
import webbrowser as wb

for url in urlsTest: 
    wb.open(url)

Le code qui précède doit avoir ouvert 10 nouveaux onglets dans le navigateur par défaut. Nous pouvons maintenant nous rendre compte que nous avons à boire et à manger. En l'occurrence, les liens peuvent se classer en deux catégories: 
<ul>
<li> liens pointant sur des posts de réseaux sociaux (e.g. twitter, mais pas seulement) <b>ne nous intéressent pas</b> (e.g. https://t.co/EbtX7NPvJb)
<li> liens pointant sur autre chose que des réseaux sociaux <b>ceux qui nous intéressent </b> (e.g. https://t.co/s6vf7joplx)
</ul>

Le problème, c'est que les urls dont nous disposons sont des urls raccourcies par le serveur de la chatbox => le <b>nom de domaine</b> n'est pas immédiatement lisible dans nos données. Il nous faut un moyen simple <b>et rapide</b> de récupérer les noms de domaines pour ne garder que ce qui nous intéresse.

Il existe une bonne librairie pour le faire en python: <a href="http://docs.python-requests.org/en/master/">requests</a>

In [ ]:
import requests
r = requests.get('https://t.co/EbtX7NPvJb')

print(r.url)
print(r.status_code)

Bon, appliquons cela sur l'intégralité de la base de données

In [ ]:
dataUrl['urls'] = dataUrl['urls'].requests.get('https://t.co/EbtX7NPvJb').url
dataUrl['urls'] = dataUrl['urls'].str.requests.get('https://t.co/EbtX7NPvJb').url

Bouh sniff, Bouh sniff, ça ne marche pas. Pourquoi? Parce que nous sommes en train de demander à python d'appliquer une méthode sur un type d'objet non prévu pour. Il nous faut penser la chose différemment avec la methode <b>apply</b>

In [ ]:
def GetRealUrl(url):           # on crée une fonction pour récupérer les urls et les éventuels codes d'erreur
    try:
        response = requests.get(url)

        # Consider any status other than 2xx an error
        if not response.status_code // 100 == 2:
            return "Error: Unexpected response {}".format(response)
        
        return response.url
    except requests.exceptions.RequestException as e:
        # A serious problem happened, like an SSLError or InvalidURL
        return "Error: {}".format(e)

urlsTest = dataUrl['urls'].sample(60).tolist()    

start = time()
None #CODE A COMPLÉTER
None #CODE A COMPLÉTER
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

Bon, nous sommes un peu dans la m*: à peu près 60s pour 60 urls => 60000 secondes pour tout le data set => 16 heures pour tout vérifier. Damned. Juste pour s'en assurrer, le problème vient-il de la boucle?

In [ ]:
dataUrlTest = dataUrl.sample(60)

start = time()
dataUrlTest['urls'] = dataUrlTest['urls'].apply(lambda x: GetRealUrl(x))
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

print(dataUrlTest['urls'])

Le gain est vraiment marginal. Ça n'a rien d'étonnant puisque le principal bottleneck, ici, ce n'est pas le <b>calcul</b> mais le <b>temps nécessaire à chaque requête</b> (envoi de la requête, réponse du serveur); paramètre sur lequel nous n'avons pas vraiment de contrôle. 

Il pourrait être pas mal de nous doter d'un moyen de passer plusieurs requêtes simultanément et ainsi ne pas avoir à attendre la réponse de la requête Rn avant d'envoyer la requête Rn+1. Il nous faut paralléliser nos opérations. 

## La parallélisation en Python: multithreading VS multiprocessing

Il faut bien distinguer deux choses.
<ul>
<li> <b>multithreading</b> : multiples opérations passées au même core CPU: ce sont les opérations de lecture et d'écriture (<i>IO operation</i>) qui sont multipliées
<li> <b>multiprocessing</b>: parallélisation au niveau du CPU lui même (usage de tous les coeurs)
</ul>

Ici, avec nos requêtes, le problème n'est pas computationnel => un simple multithreading ferait le boulot. Toutefois, la méthodologie de la librairie multi-processing de python s'est considérablement simplifié ces dernières années au point qu'il est devenu plus simple de l'utiliser dans tous les contextes. (voir <a href="http://chriskiehl.com/article/parallelism-in-one-line/"> cet article </a> à titre de comparaison avec les exemples que l'on retrouve traditionnellement).  Sachant que l'ordre d'exécution et la synchronisation ne nous intéresse pas, nous allons nous contenter d'une simple <b>pool</b> et d'un mapping. 

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool 

dataUrlTest = dataUrl.sample(60) # on réinitialise (histoire de ne pas travailler sur des données déjà rafinées)
pool = ThreadPool(4) # on crée 4 tâches parallèles

None #CODE A COMPLÉTER
None #CODE A COMPLÉTER
pool.close() 
pool.join()

end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)
print(dataUrlTest['urls'])

That was impressive: on passe de 60 secondes à un peu plus de 5 secondes pour 60 urls. Naivement, on peut s'attendre à 5000 secondes => une petite heure et demi pour tout vérifier. Peut-on faire encore mieux?

Dépend des cas: ici, clairement! On pourrait penser que le nombre optimal de <b>Thread</b> à mettre dans la <b>pool</b> correspond au <b>nombre de coeurs du processeur</b>. Tel serait le cas pour une tâche computationnellement plus lourde (nous serons amené à y revenir) mais pas ici. 

In [ ]:
dataUrlTest = dataUrl.sample(20000) # on réinitialise (histoire de ne pas travailler sur des données déjà rafinées)

pool = ThreadPool(30) # on crée 30 tâches parallèles

start = time()
dataUrlTest['urls'] = None #CODE A COMPLÉTER
pool.close() 
pool.join()

end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)
print(dataUrlTest['urls'])

Maintenant, on vire les "Error" et les liens pointant vers les réseaux sociaux

In [ ]:
dataUrlTest2 = dataUrlTest

dataUrlTest2 = dataUrlTest2[~dataUrlTest2["urls"].str.startswith("Error")]
dataUrlTest2 = dataUrlTest2[~dataUrlTest2["urls"].str.startswith("https://twitter.com")]
dataUrlTest2 = dataUrlTest2[~dataUrlTest2["urls"].str.startswith("https://www.facebook.com")]
dataUrlTest2 = dataUrlTest2[~dataUrlTest2["urls"].str.startswith("https://www.instagram.com")]
dataUrlTest2 = dataUrlTest2[~dataUrlTest2["urls"].str.startswith("https://www.pinterest.com")]



#dataUrlTest = dataUrlTest[~dataUrlTest.urls.str.startswith('Error')]

#dataUrlTest
print(dataUrlTest2.head(15))

print(dataUrlTest2.count)



In [ ]:
dataUrlTest2.to_json("talkboxUrlRefined.json")

## Le parsing, enfin

In [3]:
dataset = pd.read_json("talkboxUrlRefined.json")
print(dataset.head())

                    created  followers  friends hashtag  liked location  \
100016  2017-02-07 13:39:27        542     1170    None    0.0     None   
100049  2017-02-07 13:37:50        542     1170   Vegan    0.0     None   
100094  2017-02-07 13:34:41        223      385    None    0.0     None   
100105  2017-02-07 13:34:04         78      221   Vegan    0.0     None   
100128  2017-02-07 13:32:16        305      507    None    0.0     None   

        retwc                                               text  \
100016      1  b'RT @VeganHow: 3 Ingredient Homemade Crunch B...   
100049      5  b"RT @veganglobalnews: Ben &amp; Jerry's just ...   
100094      0  b'I liked a @YouTube video https://t.co/xxcXqI...   
100105      5  b"RT @veganglobalnews: Ben &amp; Jerry's just ...   
100128      0  b"One of our lovely chef's,  Ben Ackland,  has...   

                                                     urls  
100016  http://paper.li/VeganHow/1397322854?read=http%...  
100049  http://www.c

In [4]:
SubsetforTest = dataset.sample(60)
Data2RunInADecentTime = dataset.sample(1000)

Prenons une des urls de notre set 

In [5]:
from urllib.request import Request, urlopen

urlTest = Request("https://beinglibertarian.com/need-fix-libertarian-national-convention/", headers = {'User-Agent': 'Mozilla/5.0'})  # un moyen un peu hacky d'éviter une erreur 403
page = None #CODE A COMPLÉTER
pageStr = None #CODE A COMPLÉTER
print(pageStr)

<!DOCTYPE html>
<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><![endif]--><!--[if IE 7]><html class="no-js lt-ie9 lt-ie8" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><![endif]--><!--[if IE 8]><html class="no-js lt-ie9" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><![endif]--><!--[if IE 9]><html class="no-js lt-ie10" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#"><![endif]--><!--[if gt IE 9]><!--><html class="no-js" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
<head>
<meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="https://beinglibertarian.com/xmlrpc.php" rel="pingback"/>
<link href="https://beinglibertarian.com/wp-content/uploads/

C'est un peu sale pour l'instant mais nous allons pouvoir exploiter, assez facilement, la structuration de la page. Nous sommes surtout intéressés par les données textuelles (titres, titres de sections (h1, h2, h3), contenu des paragraphes), les images, la longueur globale du contenu. En outre, il pourrait être intéressant de récupérer la langue pour la suite (et s'éviter de faire du NLP sur autre chose que de l'anglais)
![HTML structure](http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png)

Commençons par voir comment récupérer ces données avec BeautifulSoup avant de concevoir notre fonction

In [ ]:
titrePrin = pageStr.title  # titre de la page
print(titrePrin)
print(type(titrePrin))

titresSect = pageStr.findAll(('h1','h2','h3','h4','h5'))
print(titresSect) # titre de la page
print(type(titresSect))

textualCont = None #CODE A COMPLÉTER
print(textualCont)
print(type(textualCont))

links= None #CODE A COMPLÉTER
print(links)

pics= None #CODE A COMPLÉTER
print(pics)

lang= None #CODE A COMPLÉTER
print(lang)

In [6]:
def ParseUrl(l):
    
    url = Request(l,headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"})  # un moyen un peu hacky d'éviter une erreur 403
    try:
        page = urlopen(url).read()
    except:
        return pd.Series(['Nan','Nan','Nan','Nan','Nan'])
    pageStr = BeautifulSoup(page, "lxml")
    titrePrin = pageStr.title
    titresSect = pageStr.findAll(('h1','h2','h3','h4','h5'))
    textualCont = pageStr.findAll('p')
    links= pageStr.findAll('a')
    pics= pageStr.findAll('img')
    return pd.Series([titrePrin,titresSect,textualCont,links,pics])

In [7]:
start = time()

a = None #CODE A COMPLÉTER # renvoie un dataframe avec les données formatées de ParseUrl
a = a.rename(columns = {0:"titre",1:"titreSect",2:"textualCont", 3:"links", 4:"pics"})
result = pd.concat None #CODE A COMPLÉTER

end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

temps nécessaire:  107.57017683982849


In [ ]:
a

In [ ]:
result # ça marche, prochaine étape, paralléliser

In [ ]:
start = time()  # à runner

a = Data2RunInADecentTime.apply( lambda x: ParseUrl(x["urls"]), axis = 1)# renvoie un dataframe
a = a.rename(columns = {0:"titre",1:"titreSect",2:"textualCont", 3:"links", 4:"pics"})
resultParsed = pd.concat([Data2RunInADecentTime, a], axis=1, join_axes=[Data2RunInADecentTime.index])

end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)

## NLP: Finally

Maintenant que nous disposons de nos données, formatées et "nettoyé", nous allons pouvoir analyser pour chaque lien le contenu sémantique des données textuelles. La librairie de référence pour le faire en python s'appelle <a href="http://www.nltk.org/"> NltK </a>. Nous n'allons ne nous contenter que de quelques opérations basiques pour commencer (le but à terme étant de réaliser des sentiment analysis)

### premier pas avec NLTK
NltK travaille sur des chaines de caractère et dispose de fonctions pour (entre autre)
<ul>
<li> tokenizer des chaines de charactères (<a href="http://www.nltk.org/api/nltk.tokenize.html">voir les méthodes</a>) 
<li> 
</ul>

Nous allons d'abord importer les corpora necessaires

In [3]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [-] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model


KeyboardInterrupt: 

Voyons maintenant ce que nous pouvons aisément faire sur un texte comme celui ci.

In [41]:
text = "Yet it was watching him, with its beautiful marred face and its cruel smile. Its bright hair gleamed in the early sunlight. Its blue eyes met his own. A sense of infinite pity, not for himself, but for the painted image of himself, came over him. It had altered already, and would alter more. Its gold would wither into gray. Its red and white roses would die. For every sin that he committed, a stain would fleck and wreck its fairness. But he would not sin. The picture, changed or unchanged, would be to him the visible emblem of conscience. He would resist temptation." # Oscar Wilde The Picture of Dorian Grey  chap 5

# La première chose à faire consiste à identifier les mots individuellement (car les espaces ne suffisent pas)

tokenized = nltk.word_tokenize(text)
print(tokenized)
print(type(tokenized))

print("#########")

from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
lem = [wnl.lemmatize(t) for t in tokenized]
print(lem)

print("#########")

tagged = nltk.pos_tag(tokenized)
print(tagged)
print(type(tokenized))


['Yet', 'it', 'was', 'watching', 'him', ',', 'with', 'its', 'beautiful', 'marred', 'face', 'and', 'its', 'cruel', 'smile', '.', 'Its', 'bright', 'hair', 'gleamed', 'in', 'the', 'early', 'sunlight', '.', 'Its', 'blue', 'eyes', 'met', 'his', 'own', '.', 'A', 'sense', 'of', 'infinite', 'pity', ',', 'not', 'for', 'himself', ',', 'but', 'for', 'the', 'painted', 'image', 'of', 'himself', ',', 'came', 'over', 'him', '.', 'It', 'had', 'altered', 'already', ',', 'and', 'would', 'alter', 'more', '.', 'Its', 'gold', 'would', 'wither', 'into', 'gray', '.', 'Its', 'red', 'and', 'white', 'roses', 'would', 'die', '.', 'For', 'every', 'sin', 'that', 'he', 'committed', ',', 'a', 'stain', 'would', 'fleck', 'and', 'wreck', 'its', 'fairness', '.', 'But', 'he', 'would', 'not', 'sin', '.', 'The', 'picture', ',', 'changed', 'or', 'unchanged', ',', 'would', 'be', 'to', 'him', 'the', 'visible', 'emblem', 'of', 'conscience', '.', 'He', 'would', 'resist', 'temptation', '.']
<class 'list'>
#########
['Yet', 'it',

Nous pouvons même nous essayer à quelques analyses lexicographiques. Une fonction beaucoup utilisée pour évaluer rapidement le message d'un text consiste à analyser l'ensemble de ses n-grams à la recherche de n-grams récurrents (aussi appelés <b>collocations</b>). (n-grams = séquence continu de n mots adjacents dans un text)

NB (ce n'est pas si fantaisiste et désuet que cela puisque <a href="http://storage.googleapis.com/books/ngrams/books/datasetsv2.html">même Google le fait sur ses livres</a>)

Nous allons utiliser directement les fonctions de recherche de collocation de NLTK 


In [25]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
bigram_measures= BigramAssocMeasures()

finder = BigramCollocationFinder.from_words(tokenized, window_size = 3) # on cherche tous les bigrams dans une fenêtre de 3 mots
best = finder.nbest(bigram_measures.pmi,20) # on prend les 20 meilleurs
print(best)

[('A', 'sense'), ('For', 'every'), ('He', 'resist'), ('It', 'altered'), ('It', 'had'), ('The', 'picture'), ('Yet', 'it'), ('Yet', 'was'), ('a', 'stain'), ('alter', 'more'), ('altered', 'already'), ('be', 'to'), ('beautiful', 'face'), ('beautiful', 'marred'), ('blue', 'eyes'), ('blue', 'met'), ('bright', 'gleamed'), ('bright', 'hair'), ('came', 'over'), ('changed', 'or')]


Ce n'est peut être pas très parlant à l'échelle d'un texte si court, choisissons quelque chose de plus long

In [29]:
from nltk.corpus import gutenberg

nouvelle = gutenberg.words('shakespeare-hamlet.txt')
nouvelle

finder = None #CODE A COMPLÉTER # on cherche tous les bigrams dans une fenêtre de 3 mots
best = None #CODE A COMPLÉTER # on prend les 20 meilleurs
print(best)

[('1599', 'Primus'), ('Absent', 'felicitie'), ('Abstracts', 'Chronicles'), ('Abuses', 'damne'), ('Adam', 'dig'), ('Adams', 'Profession'), ('Adieu', 'Thine'), ('Adieu', 'euermore'), ('Admit', 'Messengers'), ('Aeneas', 'Dido'), ('Alarme', 'bedded'), ('Almost', 'Ielly'), ('Amber', 'Plum'), ('Among', 'Minerall'), ('Annuall', 'Fee'), ('Anticke', ':)'), ('Antike', 'Roman'), ('Apparell', 'proclaimes'), ('Armours', 'forg'), ('Article', 'designe')]


Nous allons plutôt nous intéresser aux analyses de sentiments avec le module sentiment de NltK

In [34]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

paragraph = "It was one of the worst movies I've seen, despite good reviews. \ Unbelievably bad acting!! Poor direction. VERY poor production. \ The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

sid = SentimentIntensityAnalyzer()
sid.polarity_scores(paragraph)

{'compound': -0.9921, 'neg': 0.609, 'neu': 0.391, 'pos': 0.0}

C'est pas mal, ça!!! Essayons avec autre chose

In [38]:
phrase1 = "VADER is smart, handsome, and funny!"
phrase2 = "VADER is VERY SMART, handsome, and FUNNY!!!"
text2 = "It was one of the worst movies I've seen, despite good reviews. VADER is VERY SMART, handsome, and FUNNY!!!"

print(sid.polarity_scores(phrase1))
print(sid.polarity_scores(phrase2))
print(sid.polarity_scores(phrase1 + " " +phrase2))
print(sid.polarity_scores(text2))

{'neg': 0.0, 'compound': 0.8439, 'pos': 0.752, 'neu': 0.248}
{'neg': 0.0, 'compound': 0.9342, 'pos': 0.767, 'neu': 0.233}
{'neg': 0.0, 'compound': 0.9693, 'pos': 0.752, 'neu': 0.248}
{'neg': 0.193, 'compound': 0.8242, 'pos': 0.391, 'neu': 0.416}


Nous disposons maintenant des quelques outils nécessaires à traiter nos données

## Application au dataset
La première étape consiste à nous doter de fonctions pour l'analyse de sentiments